# Arduino Serial → Python DAQ
Live plot + CSV logging using **pyserial + matplotlib**.

**Workflow**
1. Upload Arduino sketch that streams one line per sample: `arduino_us,raw0,raw1,...`. [Recommended Sketch Here](https://github.com/mrsoltys/GEEN3853-Data-Acquisition/blob/main/Arduino_Uno_to_Python_DAQ/Arduino_Uno_to_Python_DAQ.ino)
2. Set `N_CHANNELS` and other settings .
3. Run the notebook to live-plot and export a CSV.

**Data format (required)**
- Timestamp first (Arduino microseconds), then `N_CHANNELS` ADC readings.
- Example (2 channels): `12345678,512,900`

## 0) Install / environment notes
If you’re using **Anaconda**, you can install the interactive matplotlib backend (recommended) once:

```bash
conda install -c conda-forge ipympl pyserial pandas matplotlib
```

Then restart the kernel.

If `%matplotlib widget` fails, try `%matplotlib notebook` (classic Notebook) or use the script-based approach.


In [ ]:
# Interactive plotting backend (best for live updates in Jupyter)
%matplotlib widget

import time
import threading
from collections import deque

import serial
from serial.tools import list_ports
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

import numpy as np

Helper functions (Hidden, but must run)

In [ ]:
# -- Functions: parse_line() and reader() ---------------------------------
def parse_line(line: str, n_channels: int):
    """Parse one line from the Arduino serial stream.

    Arduino output may include:
      READY
      arduino_us,raw0,raw1,...  (header)
      <arduino_us>,<raw0>,<raw1>,...
      DONE

    Data lines must have:
      1 timestamp field + N_CHANNELS ADC fields

    Returns:
      ("DATA", arduino_us:int, raws:list[int]) or
      ("READY"/"HEADER"/"DONE", None, None) or
      None (ignore the line)
    """
    s = line.strip()
    if not s:
        return None

    sl = s.lower()

    if sl == "ready":
        return ("READY", None, None)

    if sl == "done":
        return ("DONE", None, None)

    if sl.startswith("arduino_us") or sl.startswith("us"):
        return ("HEADER", None, None)

    parts = s.split(",")

    # Require: timestamp + N_CHANNELS values
    if len(parts) < (1 + n_channels):
        return None

    try:
        arduino_us = int(parts[0].strip())
        raws = [int(p.strip()) for p in parts[1:]]
    except ValueError:
        return None

    # Enforce exact channel count (prevents silent column shifts)
    if len(raws) != n_channels:
        return None

    return ("DATA", arduino_us, raws)


def reader():
    """Background thread: read serial lines and log exactly n_samples DATA samples."""
    global stop_flag

    t0_arduino_us = None

    while not stop_flag and len(log_rows) < n_samples:
        line = ser.readline().decode("utf-8", errors="ignore")
        parsed = parse_line(line, N_CHANNELS)
        if parsed is None:
            continue

        kind, arduino_us, raws = parsed

        if kind in ("READY", "HEADER"):
            continue

        if kind == "DONE":
            stop_flag = True
            break

        # DATA
        if t0_arduino_us is None:
            t0_arduino_us = arduino_us

        t_s = (arduino_us - t0_arduino_us) / 1_000_000.0
        volts = [r * (v_ref / adc_max) for r in raws]

        # Live-plot buffers
        tbuf.append(t_s)
        for ch in range(N_CHANNELS):
            vbufs[ch].append(volts[ch])

        # Row for CSV / DataFrame
        row = {
            "pc_time_s": time.time() - t0_pc,
            "arduino_us": arduino_us,
            "t_s": t_s,
        }
        for ch in range(N_CHANNELS):
            row[f"raw_{ch}"] = raws[ch]
            row[f"volts_{ch}"] = volts[ch]

        log_rows.append(row)

    stop_flag = True

    # --- Sampling statistics (Arduino timestamps) ---
    if len(log_rows) >= 2:
        arduino_us_arr = np.array([r["arduino_us"] for r in log_rows], dtype=np.int64)
        dt_s = np.diff(arduino_us_arr) * 1e-6

        median_dt_s = float(np.median(dt_s))
        stdev_dt_s  = float(np.std(dt_s, ddof=1)) if len(dt_s) > 1 else 0.0
        median_freq_hz = 1.0 / median_dt_s if median_dt_s > 0 else float("nan")

        print("=== Sampling statistics ===")
        print(f"Samples collected: {len(arduino_us_arr)}")
        print(f"        Median Δt: {median_dt_s:.6f} s")
        print(f"         Δt stdev: {stdev_dt_s:.6f} s")
        print(f" Median frequency: {median_freq_hz:.1f} Hz")

## 1) List available serial ports
Unplug/replug the Arduino if you’re unsure which port is yours (the new port that appears is usually the Arduino).


In [ ]:
ports = list(list_ports.comports())
if not ports:
    print("No serial ports found.")
else: 
    print("Detected serial ports:")
    for i, p in enumerate(ports):
        desc = (p.description or "").strip()
        manuf = (p.manufacturer or "").strip()
        print(f"  [{i}] {p.device:>12}  | {desc}  | {manuf} ")

## 2) Connect to the Arduino (EDIT THIS CODE)
Edit `PORT` to match your Arduino port from the list above.

- macOS often looks like: `/dev/cu.usbmodemXXXX`
- Windows often looks like: `COM5`

Also check that `BAUD` matches the baud rate specified in `Serial.begin()` in your Arduino sketch

Be sure to set approprate settings for your device, like v_ref and adc_max


In [ ]:
# =========================
# EDIT Here 
# =========================
PORT = "/dev/cu.usbserial-110"   # e.g., "COM5" on Windows
BAUD = 115200                  # must match your Arduino sketch

# Acquisition settings (used later)
duration_s = 15.0
sample_rate_hz = 400
plot_window_s = 1.0

# ADC scaling (Arduino Uno)
v_ref = 5.0            #5V on Arduino Uno, 3.3V on Teensy4.1
adc_max = 1023         #2^10 on Arduino Uno, 2^12 on Teensy4.1

# Channels
N_CHANNELS = 1
CHANNEL_LABELS = ["Light"]  # must match the Arduino output order

# -------------------------
# Quick connection test
# -------------------------
ser = serial.Serial(PORT, BAUD, timeout=1)
time.sleep(2.0)   # allow Arduino reset to finish
ser.reset_input_buffer()

print(f"Connected to {PORT} at {BAUD} baud")
print("Testing communication...")

# Send a short START command so you can see a few lines come back
test_samples = 5
cmd = f"START,{sample_rate_hz},{test_samples}\n"
ser.write(cmd.encode("utf-8"))
ser.flush()
print("Sent:", cmd.strip())

for _ in range(test_samples + 5):
    line = ser.readline().decode("utf-8", errors="ignore")
    parsed = parse_line(line, N_CHANNELS)
    if parsed is None:
        continue

    kind, arduino_us, raws = parsed
    print(f"{kind}: {arduino_us}, {raws}")

ser.close()

## 4) Start acquisition (background thread)
This cell starts a background reader thread that:
- reads serial lines (`us,raw`)
- fills plotting buffers (last *N* points)
- logs the full run into `log_rows` for CSV export

If your Arduino prints a header like `us,raw`, it will be ignored automatically.


In [ ]:
# -------------------------
# Run acquisition + live plot
# -------------------------

# Derived values
n_samples = int(round(sample_rate_hz * duration_s))
N = int(sample_rate_hz * plot_window_s)  # live-plot buffer length (samples)

# State / buffers
tbuf = deque(maxlen=N)
vbufs = [deque(maxlen=N) for _ in range(N_CHANNELS)]
log_rows = []
stop_flag = False
t0_pc = time.time()

# Connect + reset (opening the port typically resets the Arduino)
ser = serial.Serial(PORT, BAUD, timeout=1)
time.sleep(2.0)
ser.reset_input_buffer()
print(f"Connected to {PORT} at {BAUD} baud")

# Tell Arduino to start streaming
cmd = f"START,{sample_rate_hz},{n_samples}\n"
ser.write(cmd.encode("utf-8"))
ser.flush()
print("Sent:", cmd.strip())
print(f"Expecting {n_samples} samples ({duration_s:.1f} s at {sample_rate_hz} Hz)")

# Start reader thread
thread = threading.Thread(target=reader, daemon=False)
thread.start()
print("Reader thread started.")

# Live plot
fig, ax = plt.subplots()
lines = []

for ch in range(N_CHANNELS):
    label = CHANNEL_LABELS[ch] if ch < len(CHANNEL_LABELS) else f"CH{ch}"
    (ln,) = ax.plot([], [], label=label)
    lines.append(ln)

ax.set_xlabel("Arduino time (s)")
ax.set_ylabel("Volts (V)")
ax.set_title("Live DAQ (Arduino → Python)")
ax.grid(True)
ax.legend(loc="upper right")

def init():
    for ln in lines:
        ln.set_data([], [])
    return lines

def update(_frame):
    if len(tbuf) < 2:
        return lines

    x = list(tbuf)
    for ch, ln in enumerate(lines):
        ln.set_data(x, list(vbufs[ch]))

    ax.relim()
    ax.autoscale_view()
    return lines

ani = FuncAnimation(
    fig,
    update,
    init_func=init,
    interval=50,
    blit=False,
    cache_frame_data=False
)

plt.show()

## 6) Stop + export to CSV ✅
Run this when you’re done collecting data.

This cell:
- stops acquisition thread
- stops animation
- closes the serial port
- closes the figure (important for Jupyter)
- writes the full log to CSV


In [ ]:
# -------------------------
# Stop acquisition + export CSV
# -------------------------

# 1) Stop acquisition thread
stop_flag = True
thread.join(timeout=1)

# 2) Stop animation (prevents the kernel from staying "busy")
try:
    ani.event_source.stop()
except Exception:
    pass

# 3) Close serial
try:
    ser.close()
except Exception:
    pass

# 4) Close figure
plt.close(fig)

# 5) Export CSV
df = pd.DataFrame(log_rows)

timestamp = time.strftime("%Y%m%d_%H%M%S")
csv_name = f"daq_data_{timestamp}.csv"
df.to_csv(csv_name, index=False)

print(f"Saved: {csv_name}")
df.head()

## 7) (Optional) Quick post-run plot
This is a static plot of the full run (after Stop + Export).


In [ ]:
if "df" in globals() and len(df) > 0:
    plt.figure()
    t = df["arduino_us"] / 1_000_000.0

    # Plot all volts_* columns that exist
    volts_cols = [c for c in df.columns if c.startswith("volts_")]
    if not volts_cols:
        print("No 'volts_*' columns found. Did you export after running acquisition?")
    else:
        for col in volts_cols:
            plt.plot(t, df[col], label=col)

        plt.xlabel("Arduino time (s)")
        plt.ylabel("Volts (V)")
        plt.title("DAQ run (static plot)")
        plt.grid(True)
        plt.legend()
        plt.show()
else:
    print("No data captured (or df not created yet). Run Stop + Export first.")